In [5]:
# General
import xarray as xr    # handles netcdf data
import numpy as np     # numerical functions etc
import pandas as pd    # dataframes
import matplotlib      # plotting functions
import matplotlib.pyplot as plt    # specific plotting function I use a lot   
from matplotlib.patches import Rectangle     # specific plotting function with an annoyingly long name
import matplotlib.gridspec as gridspec #to define the sizes of plots specifically
import geopandas as gpd    # shapefiles
import cartopy     # plotting geographical features
from shapely.geometry import MultiPolygon, Polygon #to deal with 3D polygons
import regionmask    # convert shapefile to binary 0,1 mask
import re     # regular expressions (replace substrings etc)
from geopy.geocoders import Nominatim    # get lat & lon coordinates from a place name

#for climate indices
import xclim.indices as xc_i
import xclim as xc

#For plotting
import matplotlib.ticker as plticker
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import nc_time_axis #to plot cf.time axis
import cftime #to plot cf.time axis

#For Statistics
import scipy.stats as stats
from scipy import optimize
from xclim.indices.stats import fit
from scipy.stats import norm, gamma
import lmoments3.distr                     # conda activate xclim; pip install git+https://github.com/OpenHydrology/lmoments3.git
glo = getattr(lmoments3.distr, "glo")

#For copulas
import sklarpy.univariate as sk_uni
from sklarpy.copulas import gh_copula
from sklarpy.multivariate import mvt_normal
from sklarpy.copulas import MarginalFitter
from sklarpy.univariate import UnivariateFitter
from sklarpy import print_full
from sklarpy.copulas import gumbel_copula
from sklarpy.copulas import frank_copula
from sklarpy.copulas import clayton_copula

#For handling errors
import sys
import warnings

## Necessary Functions:

In [6]:
def get_SPEI_Duration_and_Severity_RunTheory(spei,threshold_drought=-0.8,threshold_minor=-1.5,threshold_combining=1):
    '''
    Summary:
    -----------
    Function which calculates the Severity-Duration pairs for a given SPEI timeseries. 
    
    Parameters:
    -----------
    spei: xarray DataArray oject
            - Timeseries containing the spei values
            
    threshold_drought: float 
            - Threshold representing the onset of a drought event
         
    threshold_minor: float 
            - Threshold representing the minium spei value for a 1-month event to be considered a drought
    
    threshold_combining: float 
            - Threshold representing the minium spei value which needs to be exceeded that two drought events which
            are separated by only one month to be considered individual events
         
    Returns:
    ----------
    Duration: Array
            - Values representing the Duration of the Drought events      
        
    Severity: Array
            - Values representing the Severity of the Drought events   
    '''
    
    #Set-up Array
    duration=[]
    severity=[]
    
    D=0
    S=0
    for t in np.arange(0,len(spei),1):
        #Drought Condition
        if spei[t]<threshold_drought:
            D=D+1
            S=S+spei[t]
            
        #To take the case into account if last value is just one month -> ongoing heatwave, 
        #only when last severity is lower than threshold_combining and severity of event larger than threshold_minor!
        elif spei[t-1]<threshold_drought and t==len(spei)-1 and spei[t]<threshold_combining and spei[t] < threshold_minor:
            D=D+1
            S=S
            
        #The condition before corrected for the last loop
        elif t!=len(spei)-1 and spei[t-1]<threshold_drought and spei[t+1]<threshold_drought and spei[t]<threshold_combining:
            D=D+1
            S=S
        
        #If the spei value is above the threshold and none of the conditions above apply that drought event is over
        else:
            duration.append(D )
            D=0
            severity.append(S)
            S=0

    #to get values if the drought ends in the loop
    duration.append(D)
    severity.append(S)
    
    #remove all durations of 1 if Severity is less than threshold (here set to: -1.5) 
    for i in np.arange(0,len(duration),1):
        if duration[i] == 1:
            if severity[i] > threshold_minor:
                duration[i] = 0
                severity[i] = 0
                
    #Remove all 0 and define severity positive
    duration = np.array(duration)
    duration = duration[duration>0]
    
    severity = np.array(severity)
    severity = -severity
    severity = severity[severity>0]      
    
    return duration, severity

### Load Regions

In [7]:
fn_tmin = '/rds/general/user/nmerz/home/MSc_Diss/data/AR6_regions/obs/era5/model_data/era5_tmin_daily_275-330E_15--60N_su.nc'
ds_tmin = xr.open_dataset(fn_tmin).sel(time=slice("1950","2023"))
era5_tmin = ds_tmin.tmin

sf_all = gpd.read_file("/rds/general/user/nmerz/home/MSc_Diss/data/AR6_regions/shapefile").to_crs(cartopy.crs.PlateCarree())
sf_SA = sf_all[sf_all['Acronym'].str.contains("NWS|NSA|NES|SAM|SWS|SES|SSA")]
rm_SA = regionmask.mask_3D_geopandas(sf_SA, era5_tmin.lon, era5_tmin.lat).squeeze(drop = True)
sf_9 = sf_all[sf_all['Acronym'].str.contains("NWS")]
rm_9 = regionmask.mask_3D_geopandas(sf_9, era5_tmin.lon, era5_tmin.lat).squeeze(drop = True)
sf_10 = sf_all[sf_all['Acronym'].str.contains("NSA")]
rm_10 = regionmask.mask_3D_geopandas(sf_10, era5_tmin.lon, era5_tmin.lat).squeeze(drop = True)
sf_11 = sf_all[sf_all['Acronym'].str.contains("NES")]
rm_11 = regionmask.mask_3D_geopandas(sf_11, era5_tmin.lon, era5_tmin.lat).squeeze(drop = True)
sf_12 = sf_all[sf_all['Acronym'].str.contains("SAM")]
rm_12 = regionmask.mask_3D_geopandas(sf_12, era5_tmin.lon, era5_tmin.lat).squeeze(drop = True)
sf_13 = sf_all[sf_all['Acronym'].str.contains("SWS")]
rm_13 = regionmask.mask_3D_geopandas(sf_13, era5_tmin.lon, era5_tmin.lat).squeeze(drop = True)
sf_14 = sf_all[sf_all['Acronym'].str.contains("SES")]
rm_14 = regionmask.mask_3D_geopandas(sf_14, era5_tmin.lon, era5_tmin.lat).squeeze(drop = True)
sf_15 = sf_all[sf_all['Acronym'].str.contains("SSA")]
rm_15 = regionmask.mask_3D_geopandas(sf_15, era5_tmin.lon, era5_tmin.lat).squeeze(drop = True)

sf_shp_arr = [sf_9,sf_10,sf_11,sf_12,sf_13,sf_14,sf_15]
region_shp_arr = np.array([rm_9,rm_10,rm_11,rm_12,rm_13,rm_14,rm_15])

## Define the main Variables

In [8]:
#Define the Aggregation Period (1, 6, 12)
M_spei = 6
#Define the region
region = 10
region_num = region - 9
region_rm = region_shp_arr[region_num]

#Define the marginal distributions for Duration and Severity
dist_D = stats.lognorm
dist_S = stats.lognorm
plotter = True #To plot the SDF-Curves for the models
return_T = np.array([10,100])

#For the bootstrapping
ci = 0.95 #CI for the intervall
num_repeat=500 #number of bootstrap sample
seed=42 #to make output the same if the code runs again

#Based on the observational results
#for M6
D_event_era5 = 12
S_event_era5 = 27.06559516233432
D_event_mswep = 9
S_event_mswep = 15.954637497385127

#Combine Results
D_event = np.ceil((D_event_era5 + D_event_mswep)/2)
S_event = (S_event_era5 + S_event_mswep)/2
print(D_event)
print(S_event)

dur_plot = D_event + 10 #defines the range of how long the plot is plotted

11.0
21.510116329859724


In [9]:
#Based on the model evaluation

#The selection for M6 and M1 is the same so M1 is used here for convenience
M1_region_9_models=np.array([None,None,2,3,4,5,6,7])
M1_region_10_models=[None,1,None,None,4,5,6,7] #Need to exclude CAN (2) as there are only 3 droughts in histnat
M1_region_11_models=[0,1,None,None,None,5,6,7]
M1_region_12_models=[0,1,2,3,4,5,6,None]
M1_region_13_models=[0,1,2,3,4,5,6,7]
M1_region_14_models=[None,None,2,3,None,5,6,7]
M1_region_15_models=[0,1,2,3,4,5,6,7]
M1_region_models=[M1_region_9_models,M1_region_10_models,M1_region_11_models,M1_region_12_models,M1_region_13_models,M1_region_14_models,M1_region_15_models]

M12_region_9_models=[None,None,2,3,4,5,6,7]
M12_region_10_models=[None,1,None,None,4,5,6,7]
M12_region_11_models=[0,1,None,None,None,5,6,7]
M12_region_12_models=[0,1,2,3,4,5,6,None]
M12_region_13_models=[0,1,2,3,None,5,6,7]
M12_region_14_models=[None,None,2,3,None,5,6,7]
M12_region_15_models=[0,1,2,None,None,5,None,7]
M12_region_models=[M12_region_9_models,M12_region_10_models,M12_region_11_models,M12_region_12_models,M12_region_13_models,M12_region_14_models,M12_region_15_models]

if M_spei == 12:
    model_mask_arr = M12_region_models[region_num]
    model_mask = np.array([x is not None for x in model_mask_arr])
else:
    model_mask_arr = M1_region_models[region_num]
    model_mask = np.array([x is not None for x in model_mask_arr])

## Load in the Data

In [10]:
dir_data = '/rds/general/user/nmerz/home/MSc_Diss/data/AR6_regions/CMIP6/'
model_names_arr = ['ACCESS-CM2/','ACCESS-ESM1-5/','CanESM5/','FGOALS-g3/','IPSL-CM6A-LR/','MIROC6/','MRI-ESM2-0/','NorESM2-LM/']

In [12]:
#Only select models which passed the evaluation
model_names_pass = [name for name, mask in zip(model_names_arr, model_mask) if mask]
model_names_pass

['ACCESS-ESM1-5/', 'IPSL-CM6A-LR/', 'MIROC6/', 'MRI-ESM2-0/', 'NorESM2-LM/']

In [15]:
#This section calculates the return periods after the bootstrapping and saves the data

#Set up array to store the return periods for all models, experiments and bootstrapping repetitions
return_time = np.zeros((len(model_names_pass),4,num_repeat))
#Loop over the models
for mod in np.arange(0,len(model_names_pass),1):
    M_spei_str = str(M_spei)
    s_str = str(region)
    model_str = model_names_pass[mod]
    
    seed_value = 42
    rng_D = np.random.RandomState(seed_value)
    rng_S = np.random.RandomState(seed_value)
    
    #histnat
    fn_spei_histnat = dir_data + model_str + "nSPEI_HG85_cal1980_2010/spei_M" + M_spei_str + "_histnat_" + s_str + ".nc"
    spei_histnat_raw = xr.open_dataset(fn_spei_histnat)
    spei_histnat = spei_histnat_raw.spei
    spei_histnat_arr = spei_histnat.to_numpy()
    time_spei_histnat_arr = spei_histnat.time.astype("datetime64[ns]").to_numpy()
    #historic
    fn_spei_historic = dir_data + model_str + "nSPEI_HG85_cal1980_2010/spei_M" + M_spei_str + "_historic_" + s_str + ".nc"
    spei_historic_raw = xr.open_dataset(fn_spei_historic)
    spei_historic = spei_historic_raw.where(spei_historic_raw['time.year'] >= 1950, drop=True).spei
    spei_historic_arr = spei_historic.to_numpy()
    time_spei_historic_arr = spei_historic.time.astype("datetime64[ns]").to_numpy()
    #ssp245
    fn_spei_ssp245 = dir_data + model_str + "nSPEI_HG85_cal1980_2010/spei_M" + M_spei_str + "_ssp245_" + s_str + ".nc"
    spei_ssp245_raw = xr.open_dataset(fn_spei_ssp245)
    spei_ssp245 = spei_ssp245_raw.spei
    spei_ssp245_arr = spei_ssp245.to_numpy()
    time_spei_ssp245_arr = spei_ssp245.time.astype("datetime64[ns]").to_numpy()
    #ssp585
    fn_spei_ssp585 = dir_data + model_str + "nSPEI_HG85_cal1980_2010/spei_M" + M_spei_str + "_ssp585_" + s_str + ".nc"
    spei_ssp585_raw = xr.open_dataset(fn_spei_ssp585)
    spei_ssp585 = spei_ssp585_raw.spei
    spei_ssp585_arr = spei_ssp585.to_numpy()
    time_spei_ssp585_arr = spei_ssp585.time.astype("datetime64[ns]").to_numpy()
    
    simulation_spei = [spei_histnat_arr,spei_historic_arr,spei_ssp245_arr,spei_ssp585_arr]
    simulation_time = [time_spei_histnat_arr,time_spei_historic_arr,time_spei_ssp245_arr,time_spei_ssp585_arr]
    
    #Loop over the experiments
    for exp in np.arange(0,4):
        exp_name = ["histnat","historic","ssp245","ssp585"]
        spei_index = simulation_spei[exp]
        simulation_time_index = simulation_time[exp]

        D_spei, S_spei = get_SPEI_Duration_and_Severity_RunTheory(spei_index)    
        Duration = pd.Series(D_spei, name='Duration')
        Severity = pd.Series(S_spei, name='Severity')
        D_S_array = pd.concat([Duration, Severity], axis=1)
        
        num_samples = len(D_S_array['Duration'])
        alpha = 1 - ci
        
        #Loop over the repetions of the bootstrapping
        for num in np.arange(0,num_repeat,1):
            D_resampled = rng_D.choice(D_S_array['Duration'],size=num_samples,replace=True)
            S_resampled = rng_S.choice(D_S_array['Severity'],size=num_samples,replace=True)

            #Fitting D
            param_D = dist_D.fit(D_resampled,floc=0)
            if dist_D.name == 'lognorm':
                fitted_D = sk_uni.lognorm.fit(D_resampled,param_D)
            if dist_D.name == 'gamma':
                fitted_D = sk_uni.gamma.fit(D_resampled,param_D)
            #Fitting S
            param_S = dist_S.fit(S_resampled,floc=0)
            if dist_S.name == 'lognorm':
                fitted_S = sk_uni.lognorm.fit(S_resampled,param_S)

            #define univariate Distribution
            marg_dist={0:fitted_D,1:fitted_S}

            #To prevent warnings of "overflow encountered in power"
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                fitted_copula_gumble = gumbel_copula.fit(
                    data=D_S_array, method='mle',
                    univariate_fitter_options={'significant': False}, show_progress=False,mdists=marg_dist)
            #Length of array
            n = len(S_resampled)
            #Timeperiod under consideration
            diff = simulation_time_index[-1] - simulation_time_index[0] #!!!!CAREFULL HERE!!!!!
            N=diff.astype('timedelta64[Y]').astype('int')+1 #As it round offs but actually it should round up if it 29.9 years
            #Define till which month should be plotted +1 because of implementation
            u1 = fitted_copula_gumble.mdists[0].cdf(D_event)
            v1 = fitted_copula_gumble.mdists[1].cdf(S_event)
            #print(fitted_copula_gumble.mdists[1])
            #print(v1)
            copula_para = fitted_copula_gumble.copula_params
            def cond_gumbel_T(x):
                C = 1/u1 * np.exp(-(((-np.log(u1))**copula_para.theta)+(-np.log(v1))**copula_para.theta)**(1/copula_para.theta))*(1+((-np.log(v1))/(-np.log(u1)))**copula_para.theta)**(-1+(1/copula_para.theta))
                return (N / (n *(1 - C))) - x
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                return_time[mod][exp][num] = optimize.root(cond_gumbel_T,u1,method='lm').x

T_histnat_mean = []
T_histnat_lq = []
T_histnat_up = []
T_historic_mean = []
T_historic_lq = []
T_historic_up = []
T_ssp245_mean = []
T_ssp245_lq = []
T_ssp245_up = []
T_ssp585_mean = []
T_ssp585_lq = []
T_ssp585_up = []

for mod in np.arange(0,len(model_names_pass),1):
    T_histnat_mean.append(np.mean(return_time[mod][0]))
    T_histnat_lq.append(np.percentile(return_time[mod][0], alpha/2*100, axis=0))
    T_histnat_up.append(np.percentile(return_time[mod][0], (1-alpha/2)*100, axis=0))
    T_historic_mean.append(np.mean(return_time[mod][1]))
    T_historic_lq.append(np.percentile(return_time[mod][1], alpha/2*100, axis=0))
    T_historic_up.append(np.percentile(return_time[mod][1], (1-alpha/2)*100, axis=0))
    T_ssp245_mean.append(np.mean(return_time[mod][2]))
    T_ssp245_lq.append(np.percentile(return_time[mod][2], alpha/2*100, axis=0))
    T_ssp245_up.append(np.percentile(return_time[mod][2], (1-alpha/2)*100, axis=0))
    T_ssp585_mean.append(np.mean(return_time[mod][3]))
    T_ssp585_lq.append(np.percentile(return_time[mod][3], alpha/2*100, axis=0))
    T_ssp585_up.append(np.percentile(return_time[mod][3], (1-alpha/2)*100, axis=0))


model_names_pass
data_T = {
    'Models': model_names_pass,
    'Mean: T histnat': T_histnat_mean,
    'LQ: T histnat': T_histnat_lq,
    'UQ: T histnat': T_histnat_up,
    'Mean: T historic': T_historic_mean,
    'LQ: T historic': T_historic_lq,
    'UQ: T historic': T_historic_up,
    'Mean: T ssp245': T_ssp245_mean,
    'LQ: T ssp245': T_ssp245_lq,
    'UQ: T ssp245': T_ssp245_up,
    'Mean: T ssp585': T_ssp585_mean,
    'LQ: T ssp585': T_ssp585_lq,
    'UQ: T ssp585': T_ssp585_up,
}
  
df_data_T = pd.DataFrame(data_T)
display(df_data_T)

,Models,Mean: T histnat,LQ: T histnat,UQ: T histnat,Mean: T historic,LQ: T historic,UQ: T historic,Mean: T ssp245,LQ: T ssp245,UQ: T ssp245,Mean: T ssp585,LQ: T ssp585,UQ: T ssp585
0,ACCESS-ESM1-5/,3220.138527,639.105402,9220.839937,2668.624598,171.484017,9637.586129,39.759594,22.145463,61.615559,63.193651,31.520836,94.510376
1,IPSL-CM6A-LR/,4460.717724,635.907339,8965.129335,1735.179667,294.839555,4389.359516,25.943853,12.541927,41.928000,32.281149,15.642352,47.116300
2,MIROC6/,2504.386645,581.178390,5317.756421,4132.376471,763.918726,9310.804738,501.764344,174.164599,729.642545,106.216772,51.966894,162.900629
3,MRI-ESM2-0/,338.869923,35.758516,1208.543216,422.571711,45.560555,1396.617754,56.433524,13.164427,138.523142,18.722808,9.299798,33.381187
4,NorESM2-LM/,10677.431042,211.278457,54459.892258,149.276835,16.555710,487.977606,21.173146,10.607338,33.207663,5.229746,3.645451,7.660458


In [16]:
# This section calculates the mean, lower and upper bound of the probability ratios

p_historic_histnat_arr=np.zeros((len(model_names_pass),len(return_time[0][0])*len(return_time[0][0])))
p_ssp245_histnat_arr=np.zeros((len(model_names_pass),len(return_time[0][0])*len(return_time[0][0])))
p_ssp585_histnat_arr=np.zeros((len(model_names_pass),len(return_time[0][0])*len(return_time[0][0])))

for mod in np.arange(0,len(model_names_pass),1):
    #ticker is needed as all combinations should be stores in one sub-array
    ticker=0
    for i in np.arange(0,len(return_time[0][0]),1):
        for j in np.arange(0,len(return_time[0][0]),1):
            p_historic_histnat_arr[mod][ticker] = return_time[mod][0][j]/return_time[mod][1][i]
            p_ssp245_histnat_arr[mod][ticker] = return_time[mod][1][j]/return_time[mod][2][i]
            p_ssp585_histnat_arr[mod][ticker] = return_time[mod][1][j]/return_time[mod][3][i]
            ticker+=1

#Prepare empty arrays
p_historic_histnat_mean = []
p_historic_histnat_lq = []
p_historic_histnat_up = []
p_ssp245_histnat_mean = []
p_ssp245_histnat_lq = []
p_ssp245_histnat_up = []
p_ssp585_histnat_mean = []
p_ssp585_histnat_lq = []
p_ssp585_histnat_up = []

#Loop over the models which pass the evaluation to add the values to the list
for mod in np.arange(0,len(model_names_pass),1):
    p_historic_histnat_mean.append(np.mean(p_historic_histnat_arr[mod]))
    p_historic_histnat_lq.append(np.percentile(p_historic_histnat_arr[mod], alpha/2*100, axis=0))
    p_historic_histnat_up.append(np.percentile(p_historic_histnat_arr[mod], (1-alpha/2)*100, axis=0))
    p_ssp245_histnat_mean.append(np.mean(p_ssp245_histnat_arr[mod]))
    p_ssp245_histnat_lq.append(np.percentile(p_ssp245_histnat_arr[mod], alpha/2*100, axis=0))
    p_ssp245_histnat_up.append(np.percentile(p_ssp245_histnat_arr[mod], (1-alpha/2)*100, axis=0))
    p_ssp585_histnat_mean.append(np.mean(p_ssp585_histnat_arr[mod]))
    p_ssp585_histnat_lq.append(np.percentile(p_ssp585_histnat_arr[mod], alpha/2*100, axis=0))
    p_ssp585_histnat_up.append(np.percentile(p_ssp585_histnat_arr[mod], (1-alpha/2)*100, axis=0))


model_names_pass
data_P = {
    'Models': model_names_pass,
    'Mean: P historic': p_historic_histnat_mean,
    'LQ: P historic': p_historic_histnat_lq,
    'UQ: P historic': p_historic_histnat_up,
    'Mean: P ssp245': p_ssp245_histnat_mean,
    'LQ: P ssp245': p_ssp245_histnat_lq,
    'UQ: P ssp245': p_ssp245_histnat_up,
    'Mean: P ssp585': p_ssp585_histnat_mean,
    'LQ: P ssp585': p_ssp585_histnat_lq,
    'UQ: P ssp585': p_ssp585_histnat_up,
}
  
df_data_P = pd.DataFrame(data_P)
display(df_data_P)  

,Models,Mean: P historic,LQ: P historic,UQ: P historic,Mean: P ssp245,LQ: P ssp245,UQ: P ssp245,Mean: P ssp585,LQ: P ssp585,UQ: P ssp585
0,ACCESS-ESM1-5/,3.888702,0.156691,22.672897,71.915936,3.957085,274.029607,45.726357,2.510540,176.373486
1,IPSL-CM6A-LR/,4.177747,0.334583,17.263863,73.624836,10.874984,217.041426,58.121161,8.750466,168.933689
2,MIROC6/,1.028748,0.113623,4.167237,9.437463,1.297895,29.501742,42.634377,6.219191,119.818781
3,MRI-ESM2-0/,1.796194,0.065468,9.954801,11.277218,0.661260,50.612321,25.138061,2.195282,90.660241
4,NorESM2-LM/,159.286025,1.385096,1048.236645,7.717885,0.734121,27.564411,29.653388,3.025195,99.997892
